<a href="https://colab.research.google.com/github/Toljatolja/Finish/blob/main/Finish_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas mapclassify  # mapclassify необходим для визуализации
!pip install geopandas
!pip install osmnx  # устанавливаем модуль osmnx
!pip install keplergl  # для визуализации

In [ ]:
# TODO собрать импорты всех модулей в одном месте
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
from keplergl import KeplerGl

In [ ]:
TILES = "CartoDB positron"  # Название подложки для карт
TERRITORY_NAME = 'Московский район, Санкт-Петербург'  # название территории для которой будут строиться слои
STREETS_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/architectural_structures.geojson"# ссылка на слой с объектами культурного наследия

In [ ]:
district = ox.geocode_to_gdf(TERRITORY_NAME)
district

In [ ]:
district.explore()

In [ ]:
osm_id = "R338636"  # https://www.openstreetmap.org/relation/338636

In [ ]:
territory = ox.geocode_to_gdf([osm_id], by_osmid=True)
territory

In [ ]:
#Загрузка слоев с улицами
file_url = STREETS_FILE_URL
district_streets = ox.geocode_to_gdf(TERRITORY_NAME)
district_streets


In [ ]:
district_streets.explore()

In [ ]:
gdf_street = gpd.read_file(file_url, mask=district_streets)
gdf_street.head()

In [ ]:
osm_id_street = "W338636"  # https://www.openstreetmap.org/relation/338636

Проверка дубликатов

In [ ]:
gdf_street.duplicated().value_counts()

Проверка дубликатов по имени name

In [ ]:
gdf_street.duplicated(["name"]).value_counts()

In [ ]:
gdf_street.drop_duplicates('name',inplace=True)

In [ ]:
gdf_street.head()

In [ ]:
gdf_street.crs

In [ ]:
gdf_street.explore(tiles=TILES)

In [ ]:
routes_graph = ox.graph_from_place(TERRITORY_NAME, network_type='all')
routes_graph


In [ ]:
roads = ox.graph_to_gdfs(routes_graph, nodes=False)  # узлы в данной задаче не нужны
roads.head()

In [ ]:
#Избавляемся от индекса
roads.reset_index(drop=True, inplace=True)
roads.head()

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=roads.copy(), name="Улицы")  # почему-то метод add_data изменяет геодатафрейм :( будем делать копию

map

Объединение геометрий


In [ ]:
roads["name"] = roads.loc[roads['name'].apply(lambda x: isinstance(x, list)), "name"].str.join(", ")
#roads

C помощью метода dissolve сгруппируем геометрии по столбцу name и объединим геометрии в одну


In [ ]:
roads = roads.dissolve(by="name")
roads.head()

In [ ]:
pharmacy = ox.features.features_from_polygon(
    territory.unary_union, {"amenity": "pharmacy"}
)
pharmacy.head()

In [ ]:
pharmacy.reset_index(drop=True, inplace=True)
pharmacy = pharmacy.loc[:, ["name", "geometry"]]
pharmacy.head()

In [ ]:
pharmacy["lon"] = pharmacy.to_crs(3857).geometry.centroid.to_crs(4326).x
pharmacy["lat"] = pharmacy.to_crs(3857).geometry.centroid.to_crs(4326).y
pharmacy.head()

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=pharmacy.copy(), name="Аптеки")  # почему-то метод add_data изменяет геодатафрейм :( будем делать копию

map

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=roads.copy(), name="Улицы")  # почему-то метод add_data изменяет геодатафрейм :( будем делать копию

map